# 入门基础：

*   GPU配置-->修改-->笔记本设置
*   查看GPU显卡信息
*   查看CPU信息
*   测试TensorFlow代码
*   安装PyTorch
*   测试PyTorch GPU代码
*   测试PyTorch CPU代码
*   管理会话
*   查看帮助


In [2]:
!nvidia-smi

Wed Oct 31 20:06:25 2018       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 396.44                 Driver Version: 396.44                    |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [3]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2300.000
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm pti ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms xsaveopt arat arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf
bogomips	: 4600.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	: 1
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
mode

In [4]:
import tensorflow as tf

print(tf.__version__)

m1 = tf.constant([[2, 2]])
m2 = tf.constant([[3],
                  [3]])

dot_operation = tf.matmul(m1, m2)

sess = tf.Session()
result = sess.run(dot_operation)
print(result)
sess.close()



1.12.0-rc2
[[12]]


In [5]:
import torch

ModuleNotFoundError: ignored

In [6]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

tcmalloc: large alloc 1073750016 bytes == 0x584d8000 @  0x7fa46af0a2a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [7]:
print(torch.__version__)

0.4.1


In [8]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
from time import time

# torch.manual_seed(1)

EPOCH = 1
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MNIST = True


cuda_use = 1

train_data = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)

# !!!!!!!! Change in here !!!!!!!!! #

if cuda_use:
    test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000].cuda()/255.   # Tensor on GPU
    test_y = test_data.test_labels[:2000].cuda()
else:
    test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000]/255.   # Tensor on CPU
    test_y = test_data.test_labels[:2000]

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2,),
                                   nn.ReLU(), nn.MaxPool2d(kernel_size=2),)
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2),)
        self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

cnn = CNN()

# !!!!!!!! Change in here !!!!!!!!! #
if cuda_use:
    cnn.cuda()      # Moves all model parameters and buffers to the GPU.

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()


for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):
        start = time()
        # !!!!!!!! Change in here !!!!!!!!! #
        if cuda_use:
            b_x = b_x.cuda()    # Tensor on GPU
            b_y = b_y.cuda()    # Tensor on GPU

        output = cnn(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            end = time()
            print('Run Tims is %.4f!' % (end-start))
            test_output = cnn(test_x)

            # !!!!!!!! Change in here !!!!!!!!! #
            if cuda_use:
                pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze()  # move the computation in GPU
            else:
                pred_y = torch.max(test_output, 1)[1].data.squeeze()

            accuracy = torch.sum(pred_y == test_y).type(torch.FloatTensor) / test_y.size(0)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.cpu().numpy(), '| test accuracy: %.3f' % accuracy)


test_output = cnn(test_x[:10])

# !!!!!!!! Change in here !!!!!!!!! #
if cuda_use:
    pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze() # move the computation in GPU
else:
    pred_y = torch.max(test_output, 1)[1].data.squeeze()

print(pred_y, 'prediction number')
print(test_y[:10], 'real number')

Processing...
Done!
Run Tims is 0.0116!
Epoch:  0 | train loss: 2.2987 | test accuracy: 0.148
Run Tims is 0.0050!
Epoch:  0 | train loss: 0.7341 | test accuracy: 0.817
Run Tims is 0.0046!
Epoch:  0 | train loss: 0.3100 | test accuracy: 0.887
Run Tims is 0.0049!
Epoch:  0 | train loss: 0.1270 | test accuracy: 0.920
Run Tims is 0.0047!
Epoch:  0 | train loss: 0.1157 | test accuracy: 0.924
Run Tims is 0.0047!
Epoch:  0 | train loss: 0.1193 | test accuracy: 0.945
Run Tims is 0.0047!
Epoch:  0 | train loss: 0.1508 | test accuracy: 0.954
Run Tims is 0.0047!
Epoch:  0 | train loss: 0.1611 | test accuracy: 0.952
Run Tims is 0.0046!
Epoch:  0 | train loss: 0.4166 | test accuracy: 0.957
Run Tims is 0.0047!
Epoch:  0 | train loss: 0.2340 | test accuracy: 0.965
Run Tims is 0.0048!
Epoch:  0 | train loss: 0.0953 | test accuracy: 0.969
Run Tims is 0.0051!
Epoch:  0 | train loss: 0.1016 | test accuracy: 0.972
Run Tims is 0.0048!
Epoch:  0 | train loss: 0.0756 | test accuracy: 0.970
Run Tims is 0.0047

In [9]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
from time import time

# torch.manual_seed(1)

EPOCH = 1
BATCH_SIZE = 50
LR = 0.001
DOWNLOAD_MNIST = True


cuda_use = 0

train_data = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=torchvision.transforms.ToTensor(), download=DOWNLOAD_MNIST,)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)

# !!!!!!!! Change in here !!!!!!!!! #

if cuda_use:
    test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000].cuda()/255.   # Tensor on GPU
    test_y = test_data.test_labels[:2000].cuda()
else:
    test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000]/255.   # Tensor on CPU
    test_y = test_data.test_labels[:2000]

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2,),
                                   nn.ReLU(), nn.MaxPool2d(kernel_size=2),)
        self.conv2 = nn.Sequential(nn.Conv2d(16, 32, 5, 1, 2), nn.ReLU(), nn.MaxPool2d(2),)
        self.out = nn.Linear(32 * 7 * 7, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        output = self.out(x)
        return output

cnn = CNN()

# !!!!!!!! Change in here !!!!!!!!! #
if cuda_use:
    cnn.cuda()      # Moves all model parameters and buffers to the GPU.

optimizer = torch.optim.Adam(cnn.parameters(), lr=LR)
loss_func = nn.CrossEntropyLoss()


for epoch in range(EPOCH):
    for step, (b_x, b_y) in enumerate(train_loader):
        start = time()
        # !!!!!!!! Change in here !!!!!!!!! #
        if cuda_use:
            b_x = b_x.cuda()    # Tensor on GPU
            b_y = b_y.cuda()    # Tensor on GPU

        output = cnn(b_x)
        loss = loss_func(output, b_y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if step % 50 == 0:
            end = time()
            print('Run Tims is %.4f!' % (end-start))
            test_output = cnn(test_x)

            # !!!!!!!! Change in here !!!!!!!!! #
            if cuda_use:
                pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze()  # move the computation in GPU
            else:
                pred_y = torch.max(test_output, 1)[1].data.squeeze()

            accuracy = torch.sum(pred_y == test_y).type(torch.FloatTensor) / test_y.size(0)
            print('Epoch: ', epoch, '| train loss: %.4f' % loss.data.cpu().numpy(), '| test accuracy: %.3f' % accuracy)


test_output = cnn(test_x[:10])

# !!!!!!!! Change in here !!!!!!!!! #
if cuda_use:
    pred_y = torch.max(test_output, 1)[1].cuda().data.squeeze() # move the computation in GPU
else:
    pred_y = torch.max(test_output, 1)[1].data.squeeze()

print(pred_y, 'prediction number')
print(test_y[:10], 'real number')

Run Tims is 0.0624!
Epoch:  0 | train loss: 2.2879 | test accuracy: 0.169
Run Tims is 0.0588!
Epoch:  0 | train loss: 0.3518 | test accuracy: 0.809
Run Tims is 0.0630!
Epoch:  0 | train loss: 0.3422 | test accuracy: 0.896
Run Tims is 0.0637!
Epoch:  0 | train loss: 0.7440 | test accuracy: 0.919
Run Tims is 0.0631!
Epoch:  0 | train loss: 0.1632 | test accuracy: 0.932
Run Tims is 0.0640!
Epoch:  0 | train loss: 0.1188 | test accuracy: 0.950
Run Tims is 0.0631!
Epoch:  0 | train loss: 0.1902 | test accuracy: 0.952
Run Tims is 0.0640!
Epoch:  0 | train loss: 0.1145 | test accuracy: 0.952
Run Tims is 0.0633!
Epoch:  0 | train loss: 0.0727 | test accuracy: 0.961
Run Tims is 0.0639!
Epoch:  0 | train loss: 0.0924 | test accuracy: 0.961
Run Tims is 0.0635!
Epoch:  0 | train loss: 0.0556 | test accuracy: 0.965
Run Tims is 0.0641!
Epoch:  0 | train loss: 0.0193 | test accuracy: 0.970
Run Tims is 0.0637!
Epoch:  0 | train loss: 0.1246 | test accuracy: 0.965
Run Tims is 0.0632!
Epoch:  0 | train 